In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from pandas_profiling import ProfileReport
# from sklearn.impute import SimpleImputer
# from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# from datetime import datetime

In [2]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [3]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [4]:
from catboost import CatBoostClassifier

In [5]:
df = pd.read_csv('US_Accidents_Dec20_with_dummies.csv')

In [6]:
df.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),...,Weather_Timestamp_hour_20,Weather_Timestamp_hour_21,Weather_Timestamp_hour_22,Weather_Timestamp_hour_23,Weather_Timestamp_weekday_1,Weather_Timestamp_weekday_2,Weather_Timestamp_weekday_3,Weather_Timestamp_weekday_4,Weather_Timestamp_weekday_5,Weather_Timestamp_weekday_6
0,3,40.108910,-83.092860,40.112060,-83.031870,3.230,42.1,36.1,58.0,29.76,...,0,0,0,0,0,0,0,0,0,0
1,3,39.172393,-84.492792,39.170476,-84.501798,0.500,37.0,29.8,93.0,29.69,...,0,0,0,0,0,0,0,0,0,0
2,2,39.063240,-84.032430,39.067310,-84.058510,1.427,35.6,29.2,100.0,29.66,...,0,0,0,0,0,0,0,0,0,0
3,2,39.067080,-84.058550,39.063020,-84.032540,1.423,35.6,29.2,100.0,29.66,...,0,0,0,0,0,0,0,0,0,0
4,2,39.775650,-84.186030,39.772750,-84.188050,0.227,33.8,57.0,100.0,29.63,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.dtypes

Severity                         int64
Start_Lat                      float64
Start_Lng                      float64
End_Lat                        float64
End_Lng                        float64
                                ...   
Weather_Timestamp_weekday_2      int64
Weather_Timestamp_weekday_3      int64
Weather_Timestamp_weekday_4      int64
Weather_Timestamp_weekday_5      int64
Weather_Timestamp_weekday_6      int64
Length: 336, dtype: object

In [7]:
df.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370980 entries, 0 to 1370979
Columns: 336 entries, Severity to Weather_Timestamp_weekday_6
dtypes: float64(25), int64(311)
memory usage: 3.4 GB


In [10]:
def downcast_columns(df):
    for col in df.columns:
        if df[col].dtypes == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [12]:
df_2 = downcast_columns(df)

In [14]:
dtypes = df_2.dtypes

colnames = dtypes.index
types = [i.name for i in dtypes.values]
column_types = dict(zip(colnames, types))

In [18]:
column_types

{'Severity': 'int8',
 'Start_Lat': 'float32',
 'Start_Lng': 'float32',
 'End_Lat': 'float32',
 'End_Lng': 'float32',
 'Distance(mi)': 'float32',
 'Temperature(F)': 'float32',
 'Wind_Chill(F)': 'float32',
 'Humidity(%)': 'float32',
 'Pressure(in)': 'float32',
 'Visibility(mi)': 'float32',
 'Wind_Speed(mph)': 'float32',
 'Precipitation(in)': 'float32',
 'Amenity': 'float32',
 'Bump': 'float32',
 'Crossing': 'float32',
 'Give_Way': 'float32',
 'Junction': 'float32',
 'No_Exit': 'float32',
 'Railway': 'float32',
 'Roundabout': 'float32',
 'Station': 'float32',
 'Stop': 'float32',
 'Traffic_Calming': 'float32',
 'Traffic_Signal': 'float32',
 'Turning_Loop': 'float32',
 'Side_R': 'int8',
 'State_AR': 'int8',
 'State_AZ': 'int8',
 'State_CA': 'int8',
 'State_CO': 'int8',
 'State_CT': 'int8',
 'State_DC': 'int8',
 'State_DE': 'int8',
 'State_FL': 'int8',
 'State_GA': 'int8',
 'State_IA': 'int8',
 'State_ID': 'int8',
 'State_IL': 'int8',
 'State_IN': 'int8',
 'State_KS': 'int8',
 'State_KY': 'i

In [15]:
df_optimized = pd.read_csv('US_Accidents_Dec20_with_dummies.csv',dtype=column_types)

In [16]:
df_optimized.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370980 entries, 0 to 1370979
Columns: 336 entries, Severity to Weather_Timestamp_weekday_6
dtypes: float32(25), int8(311)
memory usage: 537.4 MB


In [17]:
df_optimized.to_csv('US_Accidents_Dec20_with_dummies_optimized.csv', index=False)

In [3]:
df = pd.read_csv('US_Accidents_Dec20_with_dummies_opt.csv')

In [4]:
df.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370980 entries, 0 to 1370979
Columns: 241 entries, Severity to End_Time_weekday_6
dtypes: float64(25), int64(216)
memory usage: 2.5 GB


In [5]:
def downcast_columns(df):
    for col in df.columns:
        if df[col].dtypes == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [6]:
df_2 = downcast_columns(df)

In [7]:
dtypes = df_2.dtypes

colnames = dtypes.index
types = [i.name for i in dtypes.values]
column_types = dict(zip(colnames, types))

In [8]:
column_types

{'Severity': 'int8',
 'Start_Lat': 'float32',
 'Start_Lng': 'float32',
 'End_Lat': 'float32',
 'End_Lng': 'float32',
 'Distance(mi)': 'float32',
 'Temperature(F)': 'float32',
 'Wind_Chill(F)': 'float32',
 'Humidity(%)': 'float32',
 'Pressure(in)': 'float32',
 'Visibility(mi)': 'float32',
 'Wind_Speed(mph)': 'float32',
 'Precipitation(in)': 'float32',
 'Amenity': 'float32',
 'Bump': 'float32',
 'Crossing': 'float32',
 'Give_Way': 'float32',
 'Junction': 'float32',
 'No_Exit': 'float32',
 'Railway': 'float32',
 'Roundabout': 'float32',
 'Station': 'float32',
 'Stop': 'float32',
 'Traffic_Calming': 'float32',
 'Traffic_Signal': 'float32',
 'Turning_Loop': 'float32',
 'Side_R': 'int8',
 'Wind_Direction_E': 'int8',
 'Wind_Direction_N': 'int8',
 'Wind_Direction_NE': 'int8',
 'Wind_Direction_NW': 'int8',
 'Wind_Direction_S': 'int8',
 'Wind_Direction_SE': 'int8',
 'Wind_Direction_SW': 'int8',
 'Wind_Direction_VAR': 'int8',
 'Wind_Direction_W': 'int8',
 'Weather_Condition_Blowing Dust / Windy':

In [3]:
df = pd.read_csv('US_Accidents_Dec20_with_dummies_CA.csv')

In [4]:
df.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406836 entries, 0 to 406835
Columns: 191 entries, Severity to End_Time_weekday_6
dtypes: float64(25), int64(166)
memory usage: 592.8 MB


In [5]:
def downcast_columns(df):
    for col in df.columns:
        if df[col].dtypes == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [6]:
df_2 = downcast_columns(df)

In [7]:
dtypes = df_2.dtypes

colnames = dtypes.index
types = [i.name for i in dtypes.values]
column_types = dict(zip(colnames, types))

In [8]:
column_types

{'Severity': 'int8',
 'Start_Lat': 'float32',
 'Start_Lng': 'float32',
 'End_Lat': 'float32',
 'End_Lng': 'float32',
 'Distance(mi)': 'float32',
 'Temperature(F)': 'float32',
 'Wind_Chill(F)': 'float32',
 'Humidity(%)': 'float32',
 'Pressure(in)': 'float32',
 'Visibility(mi)': 'float32',
 'Wind_Speed(mph)': 'float32',
 'Precipitation(in)': 'float32',
 'Amenity': 'float32',
 'Bump': 'float32',
 'Crossing': 'float32',
 'Give_Way': 'float32',
 'Junction': 'float32',
 'No_Exit': 'float32',
 'Railway': 'float32',
 'Roundabout': 'float32',
 'Station': 'float32',
 'Stop': 'float32',
 'Traffic_Calming': 'float32',
 'Traffic_Signal': 'float32',
 'Turning_Loop': 'float32',
 'Side_R': 'int8',
 'Wind_Direction_E': 'int8',
 'Wind_Direction_N': 'int8',
 'Wind_Direction_NE': 'int8',
 'Wind_Direction_NW': 'int8',
 'Wind_Direction_S': 'int8',
 'Wind_Direction_SE': 'int8',
 'Wind_Direction_SW': 'int8',
 'Wind_Direction_VAR': 'int8',
 'Wind_Direction_W': 'int8',
 'Weather_Condition_Blowing Dust / Windy':

In [11]:
df_ca_opt = pd.read_csv('US_Accidents_Dec20_with_dummies_CA.csv', dtype=column_types)

In [12]:
df_ca_opt.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406836 entries, 0 to 406835
Columns: 191 entries, Severity to End_Time_weekday_6
dtypes: float32(25), int8(166)
memory usage: 103.2 MB


In [15]:
df = pd.read_csv('US_Accidents_Dec20_with_dummies_undersampling.csv')

In [16]:
df.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99544 entries, 0 to 99543
Columns: 139 entries, Severity to Start_Time_weekday_6
dtypes: float64(25), int64(114)
memory usage: 105.6 MB


In [17]:
df.shape

(99544, 139)

In [18]:
def downcast_columns(df):
    for col in df.columns:
        if df[col].dtypes == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [19]:
df_2 = downcast_columns(df)

In [20]:
dtypes = df_2.dtypes

colnames = dtypes.index
types = [i.name for i in dtypes.values]
column_types = dict(zip(colnames, types))

In [21]:
column_types

{'Severity': 'int8',
 'Start_Lat': 'float32',
 'Start_Lng': 'float32',
 'End_Lat': 'float32',
 'End_Lng': 'float32',
 'Distance(mi)': 'float32',
 'Temperature(F)': 'float32',
 'Wind_Chill(F)': 'float32',
 'Humidity(%)': 'float32',
 'Pressure(in)': 'float32',
 'Visibility(mi)': 'float32',
 'Wind_Speed(mph)': 'float32',
 'Precipitation(in)': 'float32',
 'Amenity': 'float32',
 'Bump': 'float32',
 'Crossing': 'float32',
 'Give_Way': 'float32',
 'Junction': 'float32',
 'No_Exit': 'float32',
 'Railway': 'float32',
 'Roundabout': 'float32',
 'Station': 'float32',
 'Stop': 'float32',
 'Traffic_Calming': 'float32',
 'Traffic_Signal': 'float32',
 'Turning_Loop': 'float32',
 'Side_R': 'int8',
 'State_AR': 'int8',
 'State_AZ': 'int8',
 'State_CA': 'int8',
 'State_CO': 'int8',
 'State_CT': 'int8',
 'State_DC': 'int8',
 'State_DE': 'int8',
 'State_FL': 'int8',
 'State_GA': 'int8',
 'State_IA': 'int8',
 'State_ID': 'int8',
 'State_IL': 'int8',
 'State_IN': 'int8',
 'State_KS': 'int8',
 'State_KY': 'i

In [22]:
df = pd.read_csv('US_Accidents_Dec20_with_dummies_undersampling.csv',dtype=column_types )

In [23]:
df.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99544 entries, 0 to 99543
Columns: 139 entries, Severity to Start_Time_weekday_6
dtypes: float32(25), int8(114)
memory usage: 20.3 MB
